# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you have executed the previous notebook, you can re-open the saved state file.

This notebook shows how to convert the pixel position of a slice to its coordinates in the atlas (forth and back).


In [2]:
# core dependencies
import os
import time
from pathlib import Path


from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba import Abba

In [3]:
# -- NOT HEADLESS
abba = Abba('Adult Mouse Brain - Allen Brain Atlas V3') # You may have to click a window!
abba.show_bdv_ui()  # creates and show a bdv view

save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state') # execute the first notebook to get this file!
abba.state_load(save_dir+"/state.json") # full absolute path needed

# ALL COMMANDS ARE PERFORMED ON SELECTED SLICES!!
# since we want to register all of them, we select all of them
abba.select_all_slices()

# change display settings : channel index, min, max
abba.change_display_settings(0, 0, 500)
abba.change_display_settings(1, 0, 1200)

[java.lang.Enum.toString] 23:30:32.474 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:34.474 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:36.474 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:38.485 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:40.494 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:42.501 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:44.514 [Timer-0] INFO sc.fiji.bdvpg.cache.BoundedLinkedHashMapGlobalCache - Cache size : 0 Mb (0 %)
[java.lang.Enum.toString] 23:30:44.532 [SciJava-6e106680-Thread-1] DEBUG sc.

In [5]:
mp = abba.mp
mp.selectSlice(mp.getSlices()) # select all slices

In [6]:
mp.deselectSlice(mp.getSlices()) # deselect all

In [7]:
mp.selectSlice(mp.getSlices().get(2)) # select the third slice

In [8]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice_0 = mp.getSlices().get(0) 
slice_1 = mp.getSlices().get(1)
slice_2 = mp.getSlices().get(2)

In [11]:

from jpype.types import JString, JArray, JDouble

# Get transformation
transform_pix_to_atlas = slice_0.getSlicePixToCCFRealTransform()

DoubleArray = JArray(JDouble)

coordInImage = DoubleArray(3)
coordInCCF = DoubleArray(3)

coordInImage[0] = 500 # X (pixel)
coordInImage[1] = 500 # Y
coordInImage[2] = 0 # Z

transform_pix_to_atlas.inverse().apply(coordInImage,coordInCCF)

print('CCF coord (mm):'+str(coordInCCF))


CCF coord (mm):[3.393072983499839, 7.484356295411038, 4.395055512552738]


In [16]:
# We can also find the coordinates of the atlas into a slice
# a coordinate in z far from 0, means that it's not in the slice, but far out

coordInCCF = DoubleArray(3)
coordInCCF[0] = 6.60 # X (pixel)
coordInCCF[1] = 4.31 # Y
coordInCCF[2] = 5.58 # Z

coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))
    


Slice[0] coord (pixel):[348.7057670533017, 235.38516607310274, 3.600667073218623]
Slice[1] coord (pixel):[361.4304422128157, 287.7613316347729, 2.810165804469322]
Slice[2] coord (pixel):[409.39554996310176, 323.2207341866975, 2.5744416742725766]
Slice[3] coord (pixel):[431.2801818083278, 299.31599447740894, 1.9917078432305715]
Slice[4] coord (pixel):[444.95026340166635, 308.7895266244058, 1.0066276332493835]
Slice[5] coord (pixel):[463.49185843127947, 326.06249706598555, 0.584678610215497]
Slice[6] coord (pixel):[483.6025521582269, 345.0469921238429, -0.001998801744556466]
Slice[7] coord (pixel):[467.01072091338955, 312.29720641340094, -0.4635978193053356]
Slice[8] coord (pixel):[497.6989764955915, 354.1903325877275, -1.0038519145691438]
Slice[9] coord (pixel):[478.26404024679647, 351.95157496712204, -1.5801448152571362]
Slice[10] coord (pixel):[498.40981900528476, 357.47374136535586, -2.2822753245082295]
Slice[11] coord (pixel):[481.02416072771405, 355.5409989716218, -2.87825034582304

In [ ]:
# The slice indexed 6 should be the one which is the nearest to the atlas point.
# Note that the values in Z are small because, in absence of 3D image, 
# each section is assumed to have a size of 1 mm